# Demo

###### Imports

In [1]:
# This Python file uses the following encoding: utf-8
from lib.predict_pdf import predict_documents_pdf, predictions_dataframe
from keras.models import model_from_json
from lib.untar import ExtractNested
from gensim.models import Doc2Vec
from functools import partial
import pandas as pd
import json
import os

Using TensorFlow backend.


###### Loading models

In [2]:
models = {}
models['gensim_model'] = Doc2Vec.load('models/gensim_model_5000.d2v')

with open('models/keras_model.json') as f:
    models['keras_model'] = model_from_json(f.read())
models['keras_model'].load_weights("models/keras_weights_5000.h5")

with open('first_5000_words.json') as f:
    models['permitted_words'] = set(json.load(f))
    
predict_pdfs = partial(predict_documents_pdf, **models)

###### Input arguments

In [3]:
pdf_folder = '../files_to_predict'
csv_out_file = '../predictions2.csv'
extract_from_tar = False

labels_map = ['NON COSTITUTIVO', 'COSTITUTIVO']

###### Tar extraction (execute only the first time if starting from a tar)

In [4]:
# Accetto solo file con dimensione < 1MB
def file_filter(f):
    return os.path.getsize(str(f.absolute()))<(1024**2)
    #return f.name.endswith(u'001.pdf') and os.path.getsize(str(f.absolute()))<(1024**2*5)

In [5]:
# Solo se si deve partire da una tar
if extract_from_tar:
    tar_root = '../prova.tar.gz'
    ExtractNested(tar_root)
    move_flattened_files(tar_root[:-7], pdf_folder, file_filter)

### Prediction

In [6]:
pdf_names = [os.path.join(pdf_folder, f) for f in os.listdir(pdf_folder)]

In [7]:
predictions, filtered_filenames = predict_pdfs(pdf_names)

../files_to_predict/4908476390001.pdf è una scansione
../files_to_predict/4908921960001.pdf è una scansione
../files_to_predict/4909311650001.pdf è una scansione
../files_to_predict/4908983690001.pdf è una scansione
../files_to_predict/4138282530001.pdf è una scansione
../files_to_predict/4137960290001.pdf è una scansione
../files_to_predict/4908312250001.pdf è una scansione
../files_to_predict/4907849390001.pdf è una scansione
../files_to_predict/4138456320001.pdf è una scansione
../files_to_predict/4137265570001.pdf è una scansione
../files_to_predict/4908190400001.pdf è una scansione
../files_to_predict/4908086720001.pdf è una scansione
../files_to_predict/4908717640001.pdf è una scansione
../files_to_predict/4137094820001.pdf è una scansione
../files_to_predict/4908064870001.pdf è una scansione
../files_to_predict/4138180200001.pdf è una scansione
../files_to_predict/4136985860001.pdf è una scansione
../files_to_predict/4138333080001.pdf è una scansione
../files_to_predict/41383147

#### Show predictions

In [8]:
for name, pred in zip(filtered_filenames, predictions):
    print os.path.basename(name)[:50].ljust(50), '\t-\t', labels_map[int(round(pred))], ' (%f)' % pred

T3LAB-INJENIA_Analisi Preliminare.pdf              	-	NON COSTITUTIVO  (0.015497)
5 Planning Robot.pdf                               	-	NON COSTITUTIVO  (0.010652)
5122462300001.pdf                                  	-	COSTITUTIVO  (0.987089)


#### Save predictions to csv

In [9]:
def basenames(files):
    return [os.path.basename(f) for f in files]

In [10]:
df = predictions_dataframe(basenames(pdf_names), basenames(filtered_filenames), predictions, csv_out_file, labels_map)
df.to_csv(csv_out_file,  sep=',', index=False, encoding='utf-8')

In [11]:
df

,Nome file,Errore,Messaggio errore,Output rete,Predizione
0,T3LAB-INJENIA_Analisi Preliminare.pdf,No,,0.0154966,NON COSTITUTIVO
1,5 Planning Robot.pdf,No,,0.0106523,NON COSTITUTIVO
2,5122462300001.pdf,No,,0.987089,COSTITUTIVO
3,4908476390001.pdf,Si,scansione,,
4,4908921960001.pdf,Si,scansione,,
5,4909311650001.pdf,Si,scansione,,
6,4908983690001.pdf,Si,scansione,,
7,4138282530001.pdf,Si,scansione,,
8,4137960290001.pdf,Si,scansione,,
9,4908312250001.pdf,Si,scansione,,
